In [29]:
import numpy as np
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm

import re
import string
import os
from sklearn.metrics.pairwise import cosine_similarity
import glob
from natsort import natsorted
import time
from dotenv import load_dotenv

load_dotenv()

google_api_key = os.getenv('google_api_key')
genai.configure(api_key=google_api_key)

In [37]:
def process(corpus):

  smaller_batches = None

  if len(corpus) > 100:
    max_size = 100
    smaller_batches = [corpus[i:i + max_size] for i in range(0, len(corpus), max_size)]
    print(len(corpus), 'corpus separated into', len(smaller_batches), 'smaller batch')

    i = 0
    y = 0
    for smaller_batch in smaller_batches:
      for text in smaller_batch:
        print(text)
        embedding = genai.embed_content(model='models/embedding-001',content=text,task_type="retrieval_document")["embedding"]
        print(embedding)
        np.save(f'temp/temp_{i}.npy', embedding)
        print(f"Finish embed corpus, batch {i}")
        i += 1

      temp_directories = natsorted(glob.glob("temp/*.npy"))
      if len(temp_directories) > 1:
        corpus = []
        for e in natsorted(glob.glob("temp/*.npy")):
          print(e)
          corpus.append(np.load(e))
          embedding = np.vstack(corpus)
          print('Success corpus append')
          os.remove(e)
        np.save(f'small_batch/corpus_dense_embeddings_{y}.npy', embedding)
        print(f"Finish embed corpus {y}")
        y += 1

  else:
    embedding = genai.embed_content(model='models/embedding-001',content=corpus,task_type="retrieval_query")["embedding"] #bisa diganti jadi "retrieval_document"

  if smaller_batches is not None:
    corpus = []
    for e in natsorted(glob.glob("small_batch/*.npy")):
      print(e)
      corpus.append(np.load(e))
      embedding = np.vstack(corpus)
      print('Success corpus append')
    np.save('corpus_dense_embeddings_google_001.npy', embedding)

  return embedding

In [52]:
df = pd.read_csv('C:/Users/Geraldus Wilsen/Documents/ProjectRekomendasiObat/data/data_obat_fix_ordered.csv')
corpus = df['summary'].to_list()
corpus_embeddings = np.load('C:/Users/Geraldus Wilsen/Documents/ProjectRekomendasiObat/data/corpus_dense_embeddings_google_001.npy')
query = 'pil untuk mengurangi gatal di kulit'
query_embeddings = process([query])
rank = cosine_similarity(query_embeddings,corpus_embeddings)
rank_dict = {i: rank[0, i] for i in range(len(rank[0]))}
dense_rank = dict(sorted(rank_dict.items(), key=lambda item: item[1], reverse=True))
corpus_id = list(dense_rank.keys())
for id in corpus_id[:10]:
  print(corpus[id])

migranal pil digunakan membantu meredakan sakit kepala
sidola kayu putih minyak sidola minyak kayu putih membantu meredakan sakit perut  perut kembung  mual  masuk angin  gatal  gatal  aromaterapi
balsem kuning cap kaki tiga balsam cajuput oil membantu meringankan pusing  sakit kepala  masuk angin  membantu meredakan gatal  gatal akibat gigitan serangga
cap kaki tiga balsam kuning balsam cajuput oil membantu meringankan pusing  sakit kepala  masuk angin  membantu meredakan gatal  gatal akibat gigitan serangga
garcia kapsul membantu tubuh menurunkan gula darah  kolesterol  teka darah  melindungi jantung  mencegah kebutaan  mencegah infeksi bakteri  virus jamur  menghambat kanker hati  kanker lambung kanker paru  melawan kanker payu dara
kulit salep acidum benzoicum untuk mengobati penyakit kulit panu  kadas  kurap  kudis  kutu air gatal  gatal disebabkan infeksi jamur
chikit teck aun botol pil pil chi kit teck aun membantu meredakan sakit perut  muntah  mabuk perjala memperbaiki nafsu m